In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

/root/miniconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
MAX_TOKEN=150
batch_size = 64
num_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device={device}")

device=cuda


In [3]:
# link: http://www.manythings.org/anki/
# 这里使用的是 中英翻译
df = pd.read_csv("./tatoeba_dataset/cmn.txt", sep='\t', header=None)
df[['eng', 'zht']] = df[[0, 1]]
df = df[['eng', 'zht']]
df = df.sample(frac=1)
max_eng_len = df['eng'].map(lambda x:len(x)).max()
max_zht_len = df['zht'].map(lambda x:len(x)).max()
print(max_eng_len, max_zht_len)
df

163 44


,eng,zht
20297,The airplane made a safe landing.,這架飛機安全著陸了。
25504,Tom is the coolest person in the world.,汤姆是世界上最酷的人。
17131,I hate terrorist organizations.,我痛恨恐怖主义组织。
748,I handled it.,我處理了。
21603,This homework is difficult for me.,這個家庭作業對我來說很困難。
...,...,...
27365,We still have many other things to discuss.,我们还有许多别的事情要讨论。
4133,No one will see us.,沒有人會看到我們。
18927,She made the same mistake again.,她又犯了同樣的錯誤了。
9001,The sheet is on the bed.,床单在床上。


In [4]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# msg = "床前明月光，疑是地上霜"
# tokenized = tokenizer(msg)
# tokenized

In [5]:
import os

hf_mirror_url = "https://hf-mirror.com"
os.environ['HF_ENDPOINT'] = hf_mirror_url
print(f"HF_ENDPOINT set to: {os.environ['HF_ENDPOINT']}\n")


HF_ENDPOINT set to: https://hf-mirror.com



In [6]:
from typing import Tuple
from pandas import DataFrame
from torch import Tensor
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader


class CMNDataset(Dataset):
    def __init__(self, df: DataFrame, tokenizer_en, tokenizer_zh, source='eng'):
        super(CMNDataset, self).__init__()
        self.src_texts = df["eng"].to_list()
        self.tgt_texts = df["zht"].to_list()
        if source != 'eng':
            self.src_texts, self.tgt_texts = self.tgt_texts, self.src_texts
            
        self.src_tokenizer = tokenizer_en
        self.tgt_tokenizer = tokenizer_zh
        self.tokenize_f = lambda tokenizer, text: tokenizer(
            text,
            max_length=MAX_TOKEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)
        
    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, index) -> Tuple[Tensor, Tensor]:
        return self.tokenize_f(self.src_tokenizer, self.src_texts[index]), self.tokenize_f(self.tgt_tokenizer, self.tgt_texts[index])

df = df.sample(frac=1)
total = len(df)

tokenizer_en = AutoTokenizer.from_pretrained("bert-base-uncased", )
tokenizer_zh = AutoTokenizer.from_pretrained("bert-base-chinese")
print(f"单词表大小，en={tokenizer_en.vocab_size}, zh={tokenizer_zh.vocab_size}")

train_ds = CMNDataset(df[: int(total * 0.8)], tokenizer_en, tokenizer_zh)
test_ds = CMNDataset(df[int(total * 0.8) :], tokenizer_en, tokenizer_zh)

train_iter = DataLoader(train_ds, batch_size, shuffle=True)
test_iter = DataLoader(test_ds, batch_size, shuffle=False)

/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


单词表大小，en=30522, zh=21128


In [7]:
len(df)

30919

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(
        self,
        tokenizer_enc,
        tokenizer_dec,
        embed_size=512,
        hidden_size=1024,
        num_layers=2,
        dropout=0.1,
    ):
        super(Seq2Seq, self).__init__()
        self.tokenizer_enc = tokenizer_enc
        self.tokenizer_dec = tokenizer_dec
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_p = dropout

        self.embedding_enc = nn.Embedding(
            num_embeddings=self.tokenizer_enc.vocab_size,
            embedding_dim=self.embed_size,
            padding_idx=tokenizer_enc.pad_token_id,
        )
        self.encoder = nn.GRU(
            self.embed_size,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            dropout=self.dropout_p if self.num_layers > 1 else 0,
        )
        self.embedding_dec = nn.Embedding(
            num_embeddings=self.tokenizer_dec.vocab_size,
            embedding_dim=self.embed_size,
            padding_idx=tokenizer_dec.pad_token_id,
        )
        self.decoder = nn.GRU(
            self.embed_size,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            dropout=self.dropout_p if self.num_layers > 1 else 0,
        )
        self.fc_out = nn.Linear(self.hidden_size, self.tokenizer_dec.vocab_size)

    def forward(self, src, tgt):
        # src shape=(bs, seq_len)

        embedded_src = self.embedding_enc(src)  # (bs, embed_size)
        _, hidden = self.encoder(embedded_src)  # (bs, hidden)
        embedded_tgt = self.embedding_dec(tgt)
        output_dec, _ = self.decoder(embedded_tgt, hidden)  #
        predictions = self.fc_out(output_dec)  # (bs, vocab_size)
        return predictions

    def generate(self, src: str, max_len=150):
        self.eval()

        src_tokens = self.tokenizer_enc(
            src,
            max_length=max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].to(device)

        # b. 编码源句子，获取上下文向量
        with torch.no_grad():
            embedded_src = self.embedding_enc(src_tokens)
            _, hidden = self.encoder(embedded_src)

        tgt_tokens = [self.tokenizer_dec.cls_token_id]

        for _ in range(max_len):
            tgt_input_tensor = torch.tensor([tgt_tokens[-1]], dtype=torch.long).unsqueeze(0).to(device)

            with torch.no_grad():
                embedded_tgt = self.embedding_dec(tgt_input_tensor)
                output_dec, hidden = self.decoder(embedded_tgt, hidden)
                prediction = self.fc_out(output_dec)

            # 贪心，从词汇表概率中选择概率最大的词
            predicted_token_id = prediction.argmax(2).item()

            if predicted_token_id == self.tokenizer_dec.sep_token_id:
                break

            tgt_tokens.append(predicted_token_id)

        return self.tokenizer_dec.decode(tgt_tokens[1:], skip_special_tokens=True)

In [10]:
from tqdm import tqdm


model = Seq2Seq(
    tokenizer_enc=tokenizer_en, tokenizer_dec=tokenizer_zh, hidden_size=768, dropout=0.2
).to(device)
print("模型总参数:", sum(p.numel() for p in model.parameters()))

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_func = torch.nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x, y in tqdm(train_iter):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        pred = model(x, y[:, :-1])

        loss_val = loss_func(pred.permute(0, 2, 1), y[:, 1:])
        loss_val.backward()

        optimizer.step()
        total_loss += loss_val.item()
    print(f"Epoch {epoch+1} finished, Average Loss: {total_loss / len(train_iter):.4f}")
    
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for x_test, y_test in test_iter:
            x_test = x_test.to(device)
            y_test = y_test.to(device)

            pred_test = model(x_test, y_test[:, :-1])
            loss_val_test = loss_func(pred_test.permute(0, 2, 1), y_test[:, 1:])
            test_loss += loss_val_test.item()

        print(f"Epoch {epoch+1} Test Loss: {test_loss / len(test_iter):.4f}")
    test_sentences = [
        "I love you.",
        "How are you?",
        "This is a book.",
        "Let's go to school.",
        "My name is Tom."
    ]
    for sentence in test_sentences:
        translation = model.generate(sentence)
        print(f"{translation}", end='\t')
    print()
    torch.save(model.state_dict(), f'seq2seq_weights_ep{epoch+1}.pth')
    

模型总参数: 55686792


100%|██████████| 387/387 [01:38<00:00,  3.94it/s]


Epoch 1 finished, Average Loss: 0.8981
Epoch 1 Test Loss: 0.3698
我 我 不 的 。	我 不 是 的 。	我 我 不 的 。	我 我 不 的 。	我 我 不 的 。	


100%|██████████| 387/387 [01:38<00:00,  3.92it/s]


Epoch 2 finished, Average Loss: 0.3587
Epoch 2 Test Loss: 0.3433
我 們 在 我 的 。	你 的 是 我 的 的 。	我 們 在 我 的 。	我 們 在 我 的 。	我 們 在 我 的 。	


...

Epoch 8 finished, Average Loss: 0.2242
Epoch 8 Test Loss: 0.2288
我 想 你 。	你 们 什 么 ？	这 是 一 个 人 。	我 们 去 学 校 。	我 的 房 子 是 汤 姆 。	


100%|██████████| 387/387 [01:39<00:00,  3.87it/s]


Epoch 9 finished, Average Loss: 0.2116
Epoch 9 Test Loss: 0.2202
我 喜 欢 你 。	你 们 怎 么 样 ？	这 是 一 个 好 的 。	我 們 去 看 電 影 。	我 的 房 子 是 汤 姆 。	


100%|██████████| 387/387 [01:39<00:00,  3.87it/s]


Epoch 10 finished, Average Loss: 0.2000
Epoch 10 Test Loss: 0.2130
我 喜 欢 你 。	你 們 的 事 情 吗 ？	这 是 一 个 好 的 。	我 們 去 学 校 。	我 的 房 子 是 汤 姆 。	
